In [3]:
%pip install easyocr

In [4]:
import os
import json
import easyocr

# Mount Google Drive terlebih dahulu jika diperlukan
from google.colab import drive
drive.mount('/content/drive')

reader = easyocr.Reader(['id', 'en'])

# Path ke folder dataset di Google Drive
image_folder = "/content/drive/My Drive/dataset/train"
output_json = "/content/drive/My Drive/dataset/train_labels.json"

result_json = []

for img_name in os.listdir(image_folder):
    img_path = os.path.join(image_folder, img_name)
    result = reader.readtext(img_path, detail=1)  # Ambil teks dengan detail (bbox)

    ocr_result = {
        "gt_parse": {
            "menu": {},
            "sub_total": {},
            "total": {}
        },
        "meta": {
            "version": "2.0.0",
            "split": "train",
            "image_id": img_name.split('.')[0],
            "image_size": {"width": 0, "height": 0}  # Perlu diisi manual jika tidak ada info
        },
        "valid_line": []
    }

    for i, (bbox, text, _) in enumerate(result):
        x1, y1 = bbox[0]
        x2, y2 = bbox[2]

        word_info = {
            "quad": {
                "x1": int(x1), "y1": int(y1),
                "x2": int(x2), "y2": int(y1),
                "x3": int(x2), "y3": int(y2),
                "x4": int(x1), "y4": int(y2),
            },
            "is_key": 0,
            "row_id": 2119000 + i,
            "text": text
        }

        valid_line = {
            "words": [word_info],
            "category": "uncategorized",
            "group_id": i + 1,
            "sub_group_id": 0
        }

        # Mengategorikan teks (bisa dikembangkan lebih lanjut dengan regex atau NLP)
        if "TOTAL" in text.upper():
            valid_line["category"] = "total.total_price"
        elif "TAX" in text.upper():
            valid_line["category"] = "sub_total.tax_price"
        elif "KEMBALI" in text.upper():
            valid_line["category"] = "total.changeprice"
        elif text.replace(".", "").isdigit():  # Deteksi angka sebagai harga
            valid_line["category"] = "menu.unitprice"

        ocr_result["valid_line"].append(valid_line)

    result_json.append(ocr_result)

# Simpan hasil ke file JSON
with open(output_json, "w", encoding="utf-8") as f:
    json.dump(result_json, f, indent=4, ensure_ascii=False)

print(f"Hasil OCR tersimpan dalam format JSON di: {output_json}")

Mounted at /content/drive
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% CompleteHasil OCR tersimpan dalam format JSON di: /content/drive/My Drive/dataset/train_labels.json
